In [ ]:
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.25)

In [ ]:
with open("mmlu_user_data_NEW.json", "r") as f:
    saved_data = json.load(f)

df = []
for user_id, user_data in saved_data.items():
    user_df = pd.read_json(user_data["user_df"])
    # user_df["comments"] = user_data["comments"]
    df.append(user_df)
df = pd.concat(df, ignore_index=True)
df["didRely"] = df.apply(
    lambda row: "Agree" if row["llm_answer"] == row["response"] else "Disagree", axis=1
)

In [ ]:
for mode in ["query", "ct", "rand"]:
    plt_df = df[df.variant == f"justMConf_mistral_{mode}"]

    fig, ax = plt.subplots(figsize=(5, 3))

    g = sns.histplot(
        plt_df,
        x="model_confidence",
        hue="didRely",
        stat="probability",
        bins=20,
        kde=True,
        ax=ax,
        palette=[sns.color_palette("Paired")[7], sns.color_palette("Paired")[3]],
        hue_order=["Disagree", "Agree"],
    )

    if mode == "query":
        ax.get_legend().set(title="", loc="upper left")
    else:
        ax.get_legend().remove()

    g.set(xlabel=r"Model Confidence ($\%$)", ylabel=r"Proportion ($\%$)")
    g.figure.show()
    # g.figure.savefig(f"user_conf_{mode}.pdf", bbox_inches="tight")